### Domain Knowledge Tabel & Relasi Setiap Tabel

Informasi umum terkait tabel

1. application_train|test:
    - Tabel utama berisi permohonan kredit
2. bureau:
    - Data kredit-klien yang dilaporkan ke lembaga keuangan lain | berkaitan dengan riwayat peminjaman di institusi lain
3. bureau_balance:
    - Data bulanan riwayat kredit pada tabel bureau
4. previous_application:
    - Data permohonan pinjaman pada Home Credit oleh klien di aplikasi utama
5. POS_CASH_Balance:
    - Data bulanan terkait pinjaman "Point-of-Sale" atau pinjama uang tunai oleh klien yang sama di Home Credit
6. credit_card_balance:
    - Data bulanan terkait saldo dan aktivitas kartu kredit yang klien punya
7. installments_payments:
    - Data history pembayaran angsuran untuk pinjaman yang telah diberikan

Bentuk relasi antar tabel

1. Tabel utama (application) merupakan data utama dengan setiap barisnya itu pengajuan kredit baru (SK_ID_CURR)
2. Dari id SK_ID_CURR ini dapat dilakukan join ke banyak tabel seperti:
    - Baris pada previous_application.csv (pinjaman sebelumnya oleh klien yang sama)
    - Baris di bureau.csv (kredit eksternal dari klien yang sama) -> dari tabel ini memungkinkan join ke bureau_balance.csv untuk data bulanan dari setiap pinjaman eksternal tersebut.
    - Baris-baris di tabel POS_CASH_balance.csv, credit_card_balance.csv, installments_payments.csv yang menggambarkan jenis kredit/saldo bulanan/pembayaran angsuran sebelumnya
3. Relasi Many-to-one untuk tabel-tabel lain terhadap tabel utama.
4. Relasi ini memungkinkan untuk mengambil beberapa tabel tertentu untuk melakukan analisis spesifik terhadap klien yang ada.

### Import Library | Merging Data 

Proses pertama akan dilalui dengan mengimport library yang diperlukan dan merging data untuk menggabungkan beberapa tabel untuk dijadikan sebagai Analytical Based Table dalam menjawab pertanyaan-pertanyaan yang ada

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

#### Mengecek informasi-informasi pada tabel utama

In [2]:
main_table = pd.read_csv('application_train.csv')
main_table.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [3]:
print(f'Jumlah baris dan kolom pada tabel utama adalah : {main_table.shape}')

Jumlah baris dan kolom pada tabel utama adalah : (307511, 122)


In [4]:
main_table.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,104582.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307509.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,134133.000000,3.068510e+05,246546.000000,151450.00000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,151450.000000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,151450.000000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,159080.000000,306490.000000,306490.000000,306490.000000,306490.000000,307510.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.00000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,-4986.120328,-2994.202373,12.061091,0.999997,0.819889,0.199368,0.998133,0.281066,0.056720,2.152665,2.052463,2.031521,12.063419,0.015144,0.050769,0.040659,0.078173,0.230454,0.179555,0.502130,5.143927e-01,0.510853,0.11744,0.088442,0.977735,0.752471,0.044621,0.078942,0.149725,0.226282,0.231894,0.066333,0.100775,0.107399,0.008809,0.028358,0.114231,0.087543,0.977065,0.759637,0.042553,0.074490,0.145193,0.222315,0.228058,0.064958,0.105645,0.105975,0.008076,0.027022,0.117850,0.087955,0.97

In [5]:
# Mengecek kolom yang memiliki missing value lebih dari 40%
missing_threshold = 0.4
missing_data = main_table.isnull().sum()
missing_data = missing_data[missing_data > missing_threshold * len(main_table)]
print(f"Jumlah kolom dengan missing value lebih dari 40%: {len(missing_data)}")
missing_data

Jumlah kolom dengan missing value lebih dari 40%: 49


OWN_CAR_AGE                     202929
EXT_SOURCE_1                    173378
APARTMENTS_AVG                  156061
BASEMENTAREA_AVG                179943
YEARS_BEGINEXPLUATATION_AVG     150007
YEARS_BUILD_AVG                 204488
COMMONAREA_AVG                  214865
ELEVATORS_AVG                   163891
ENTRANCES_AVG                   154828
FLOORSMAX_AVG                   153020
FLOORSMIN_AVG                   208642
LANDAREA_AVG                    182590
LIVINGAPARTMENTS_AVG            210199
LIVINGAREA_AVG                  154350
NONLIVINGAPARTMENTS_AVG         213514
NONLIVINGAREA_AVG               169682
APARTMENTS_MODE                 156061
BASEMENTAREA_MODE               179943
YEARS_BEGINEXPLUATATION_MODE    150007
YEARS_BUILD_MODE                204488
COMMONAREA_MODE                 214865
ELEVATORS_MODE                  163891
ENTRANCES_MODE                  154828
FLOORSMAX_MODE                  153020
FLOORSMIN_MODE                  208642
LANDAREA_MODE            

#### Proses merging pada beberapa tabel yang memiliki informasi untuk menjawab pertanyaan bisnis

##### Persiapan merging tabel bureau 

In [6]:
bureau = pd.read_csv('bureau.csv')
print(f'Jumlah baris dan kolom pada tabel bureau adalah : {bureau.shape}')
bureau.head()

Jumlah baris dan kolom pada tabel bureau adalah : (1716428, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [7]:
bureau.duplicated().sum()

0

In [8]:
bureau.isna().sum()

SK_ID_CURR                      0
SK_ID_BUREAU                    0
CREDIT_ACTIVE                   0
CREDIT_CURRENCY                 0
DAYS_CREDIT                     0
CREDIT_DAY_OVERDUE              0
DAYS_CREDIT_ENDDATE        105553
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_MAX_OVERDUE    1124488
CNT_CREDIT_PROLONG              0
AMT_CREDIT_SUM                 13
AMT_CREDIT_SUM_DEBT        257669
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_OVERDUE          0
CREDIT_TYPE                     0
DAYS_CREDIT_UPDATE              0
AMT_ANNUITY               1226791
dtype: int64

In [9]:
bureau['CREDIT_ACTIVE'].value_counts()

CREDIT_ACTIVE
Closed      1079273
Active       630607
Sold           6527
Bad debt         21
Name: count, dtype: int64

In [10]:
bureau['CREDIT_TYPE'].value_counts()

CREDIT_TYPE
Consumer credit                                 1251615
Credit card                                      402195
Car loan                                          27690
Mortgage                                          18391
Microloan                                         12413
Loan for business development                      1975
Another type of loan                               1017
Unknown type of loan                                555
Loan for working capital replenishment              469
Cash loan (non-earmarked)                            56
Real estate loan                                     27
Loan for the purchase of equipment                   19
Loan for purchase of shares (margin lending)          4
Mobile operator loan                                  1
Interbank credit                                      1
Name: count, dtype: int64

Melakukan penanganan terhadap missing value pada bureau

In [7]:
for col in ["AMT_CREDIT_SUM_DEBT", "AMT_CREDIT_SUM_OVERDUE", "AMT_CREDIT_SUM_LIMIT"]:
    bureau[col].fillna(0, inplace=True)

bureau["DAYS_CREDIT_ENDDATE"].fillna(bureau["DAYS_CREDIT_ENDDATE"].median(), inplace=True)
bureau["DAYS_ENDDATE_FACT"].fillna(0, inplace=True)

C:\Users\hardi\AppData\Local\Temp\ipykernel_18644\2756992705.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  bureau[col].fillna(0, inplace=True)
C:\Users\hardi\AppData\Local\Temp\ipykernel_18644\2756992705.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when do

In [8]:
# RATIO_DEBT_TO_CREDIT: biarkan NaN jika AMT_CREDIT_SUM = 0 agar mean() mengabaikan NaN (selaras dengan BigQuery AVG mengabaikan NULL)
bureau["RATIO_DEBT_TO_CREDIT"] = bureau["AMT_CREDIT_SUM_DEBT"] / bureau["AMT_CREDIT_SUM"].replace(0, np.nan)

Alasan penanganan missing value dengan cara yang ada:
- Terdapat kolom kosong yang tidak berarti error, namun karena atributnya tidak relevan dengan tipe kredit tertentu.
- fillna(0) merepresentasikan “tidak ada kewajiban” sedangkan flag missing memberi tahu model “awalnya kolom ini kosong” — ini mencegah model kehilangan sinyal bahwa absen data itu sendiri bermakna.
- Model kredit sering memanfaatkan indikator missingness (mis. flag default) karena ketidakhadiran fitur dapat mencerminkan produk/kontrak yang berbeda dan perilaku pembayaran berbeda.

Alasan penambahan fitur RATIO_DEBT_TO_CREDIT:
- Dalam literatur kredit, rasio-rasio (Debt-to-Income, Debt-to-Exposure) adalah driver utama probability of default (PD). Semakin tinggi proporsi utang terhadap total kredit, semakin besar tekanan likuiditas yang menaikkan risiko gagal bayar.
- Rasio bersifat scale-invariant → berguna ketika jumlah kredit bervariasi besar antar nasabah; membuat fitur menjadi lebih comparable.

Melakukan penambahan kolom berdasarkan jumlah status credit yang dimiliki beserta rasio terhadap bad debt

In [9]:
credit_active_dummies = pd.get_dummies(bureau["CREDIT_ACTIVE"], prefix="STATUS")
bureau_active_agg = credit_active_dummies.groupby(bureau["SK_ID_CURR"]).sum()

# Rename agar match output ETL/BigQuery
bureau_active_agg = bureau_active_agg.rename(columns={
    "STATUS_Active": "STATUS_ACTIVE",
    "STATUS_Closed": "STATUS_CLOSED",
    "STATUS_Bad debt": "STATUS_BAD_DEBT",
})

# Pastikan kolom selalu ada (selaras dengan SUM(CASE WHEN...) di SQL)
for c in ["STATUS_ACTIVE", "STATUS_CLOSED", "STATUS_BAD_DEBT"]:
    if c not in bureau_active_agg.columns:
        bureau_active_agg[c] = 0

# TOTAL_CREDITS = COUNT(*)
bureau_active_agg["TOTAL_CREDITS"] = bureau.groupby("SK_ID_CURR").size()

# RATIO_BAD_TO_TOTAL = STATUS_BAD_DEBT / TOTAL_CREDITS
bureau_active_agg["RATIO_BAD_TO_TOTAL"] = bureau_active_agg["STATUS_BAD_DEBT"] / bureau_active_agg["TOTAL_CREDITS"]

bureau_active_agg.reset_index(inplace=True)

Alasan dibuatnya kolom baru untuk menampung jumlah status credit activenya:
- Banyak studi dan praktik bank menyatakan bahwa jumlah pinjaman aktif, jumlah pinjaman macet sebelumnya, dan proporsi macet terhadap total adalah indikator prediktif kuat untuk PD.

Melakukan pengelompokan terhadap credit type ke beberapa jenis

In [10]:
mapping = {
    'Consumer credit': 'Retail',
    'Credit card': 'Retail',
    'Cash loan (non-earmarked)': 'Retail',
    'Microloan': 'Retail',
    'Car loan': 'Asset',
    'Mortgage': 'Asset',
    'Real estate loan': 'Asset',
    'Loan for business development': 'Business',
    'Loan for working capital replenishment': 'Business',
    'Loan for purchase of shares (margin lending)': 'Business',
    'Loan for the purchase of equipment': 'Business'
}

bureau['CREDIT_GROUP'] = bureau['CREDIT_TYPE'].map(mapping).fillna('Other')

credit_group_dummies = pd.get_dummies(bureau["CREDIT_GROUP"], prefix="CREDIT_GROUP")
bureau_credit_group = credit_group_dummies.groupby(bureau["SK_ID_CURR"]).sum()

# Rename agar match output ETL/BigQuery (uppercase)
bureau_credit_group = bureau_credit_group.rename(columns={
    "CREDIT_GROUP_Retail": "CREDIT_GROUP_RETAIL",
    "CREDIT_GROUP_Asset": "CREDIT_GROUP_ASSET",
    "CREDIT_GROUP_Business": "CREDIT_GROUP_BUSINESS",
})

# Pastikan kolom selalu ada
for c in ["CREDIT_GROUP_RETAIL", "CREDIT_GROUP_ASSET", "CREDIT_GROUP_BUSINESS"]:
    if c not in bureau_credit_group.columns:
        bureau_credit_group[c] = 0

denom = bureau.groupby("SK_ID_CURR").size()
denom = denom.reindex(bureau_credit_group.index)
bureau_credit_group["RATIO_ASSET_TO_TOTAL"] = bureau_credit_group["CREDIT_GROUP_ASSET"] / denom.values

bureau_credit_group.reset_index(inplace=True)

Alasan pengkategorian ke beberapa grup untuk credit type ini:
- Karena memiliki banyak kategori (beberapa sangat jarang). One-hot penuh → meningkatkan dimensi (sparsity), overfitting, dan membuat model kurang stabil.
- Banyak kategori dapat dikelompokkan berdasarkan karakteristik risiko seperti Retail (konsumsi kecil) cenderung berbeda perilaku daripada Asset (hipotek, mobil) yang biasanya secured dan tenor panjang.

In [11]:
num_cols = ["AMT_CREDIT_SUM", "AMT_CREDIT_SUM_DEBT", "DAYS_CREDIT", "DAYS_CREDIT_ENDDATE", "RATIO_DEBT_TO_CREDIT"]

bureau_num_agg = bureau.groupby("SK_ID_CURR")[num_cols].agg(['mean'])
bureau_num_agg.columns = ["_".join(col).upper() for col in bureau_num_agg.columns]
bureau_num_agg.reset_index(inplace=True)

In [12]:
bureau_summary = (
    bureau_active_agg
    .merge(bureau_credit_group, on="SK_ID_CURR", how="left")
    .merge(bureau_num_agg, on="SK_ID_CURR", how="left")
)

In [14]:
selected_features = [
    "SK_ID_CURR",
    "STATUS_ACTIVE", "STATUS_CLOSED", "STATUS_BAD_DEBT",
    "TOTAL_CREDITS", "RATIO_BAD_TO_TOTAL",
    "CREDIT_GROUP_RETAIL", "CREDIT_GROUP_ASSET", "CREDIT_GROUP_BUSINESS",
    "RATIO_ASSET_TO_TOTAL",
    "AMT_CREDIT_SUM_MEAN", "AMT_CREDIT_SUM_DEBT_MEAN", "RATIO_DEBT_TO_CREDIT_MEAN",
    "DAYS_CREDIT_MEAN", "DAYS_CREDIT_ENDDATE_MEAN",
 ]
bureau_summary = bureau_summary[selected_features]

print(f"bureau_summary shape: {bureau_summary.shape}")
bureau_summary.head()

bureau_summary shape: (305811, 15)


,SK_ID_CURR,STATUS_ACTIVE,STATUS_CLOSED,STATUS_BAD_DEBT,TOTAL_CREDITS,RATIO_BAD_TO_TOTAL,CREDIT_GROUP_RETAIL,CREDIT_GROUP_ASSET,CREDIT_GROUP_BUSINESS,RATIO_ASSET_TO_TOTAL,AMT_CREDIT_SUM_MEAN,AMT_CREDIT_SUM_DEBT_MEAN,RATIO_DEBT_TO_CREDIT_MEAN,DAYS_CREDIT_MEAN,DAYS_CREDIT_ENDDATE_MEAN
0,100001,3,4,0,7,0.0,7,0,0,0.0,207623.571429,85240.928571,0.282518,-735.000000,82.428571
1,100002,2,6,0,8,0.0,8,0,0,0.0,108131.945625,30722.625000,0.078026,-874.000000,-344.250000
2,100003,1,3,0,4,0.0,4,0,0,0.0,254350.125000,0.000000,0.000000,-1400.750000,-544.500000
3,100004,0,2,0,2,0.0,2,0,0,0.0,94518.900000,0.000000,0.000000,-867.000000,-488.500000
4,100005,2,1,0,3,0.0,3,0,0,0.0,219042.000000,189469.500000,0.601256,-190.666667,439.333333


Alasan pemilihan-pemilihan fitur ini:
1. Cover key dimensions of credit risk: fitur yang dipilih mewakili dimensi utama PD / risk assessment:
    - Exposure quantity (TOTAL_CREDITS)
    - Exposure quality (STATUS_Bad debt, RATIO_BAD_TO_TOTAL)
    - Credit mix/type (CREDIT_GROUP_*)
    - Leverage (AMT_CREDIT_SUM_DEBT_MEAN, RATIO_DEBT_TO_CREDIT_MEAN)
    - Recency/age (DAYS_CREDIT_MEAN, DAYS_CREDIT_ENDDATE_MEAN) Ini sejajar dengan konsep PD, LGD, EAD di credit risk modeling: PD dipengaruhi oleh perilaku (status), LGD/EAD berkaitan dengan exposure dan type.

2. Mengurangi multicollinearity & redundansi: banyak agregat yang dihasilkan (sum/mean/max/min) saling berkorelasi tinggi. Memilih satu statistik representatif (mean) mengurangi redundancy.

3. Praktik industri: model scoring banking sering disusun dari lusinan fitur inti yang bisa dijelaskan; sisanya digunakan untuk eksperimen atau feature engineering lanjutan.

##### Persiapan merging previous_application

Dilakukan persiapan pada tiga tabel untuk dimerging pada tabel previous_application

In [15]:
pos_cash = pd.read_csv('POS_CASH_balance.csv')
print(f'Jumlah baris dan kolom pada tabel POS_CASH_balance adalah : {pos_cash.shape}')
pos_cash.head()

Jumlah baris dan kolom pada tabel POS_CASH_balance adalah : (10001358, 8)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [19]:
pos_cash.duplicated().sum()

0

In [20]:
pos_cash.isna().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
MONTHS_BALANCE               0
CNT_INSTALMENT           26071
CNT_INSTALMENT_FUTURE    26087
NAME_CONTRACT_STATUS         0
SK_DPD                       0
SK_DPD_DEF                   0
dtype: int64

In [21]:
pos_cash['NAME_CONTRACT_STATUS'].value_counts()

NAME_CONTRACT_STATUS
Active                   9151119
Completed                 744883
Signed                     87260
Demand                      7065
Returned to the store       5461
Approved                    4917
Amortized debt               636
Canceled                      15
XNA                            2
Name: count, dtype: int64

In [16]:
installments_payments = pd.read_csv('installments_payments.csv')
print(f'Jumlah baris dan kolom pada tabel installments_payments adalah : {installments_payments.shape}')
installments_payments.head()

Jumlah baris dan kolom pada tabel installments_payments adalah : (13605401, 8)


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [23]:
installments_payments.duplicated().sum()

0

In [24]:
installments_payments.isna().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
NUM_INSTALMENT_VERSION       0
NUM_INSTALMENT_NUMBER        0
DAYS_INSTALMENT              0
DAYS_ENTRY_PAYMENT        2905
AMT_INSTALMENT               0
AMT_PAYMENT               2905
dtype: int64

In [25]:
installments_payments['NUM_INSTALMENT_VERSION'].value_counts()

NUM_INSTALMENT_VERSION
1.0      8485004
0.0      4082498
2.0       620283
3.0       237063
4.0        55274
          ...   
178.0          1
58.0           1
54.0           1
53.0           1
57.0           1
Name: count, Length: 65, dtype: int64

In [26]:
installments_payments['NUM_INSTALMENT_NUMBER'].value_counts()

NUM_INSTALMENT_NUMBER
1      1004160
2       985716
3       968279
4       943502
5       880007
        ...   
275          2
269          2
277          1
274          1
276          1
Name: count, Length: 277, dtype: int64

In [17]:
credit_card_balance = pd.read_csv('credit_card_balance.csv')
print(f'Jumlah baris dan kolom pada tabel credit_card_balance adalah : {credit_card_balance.shape}')
credit_card_balance.head()

Jumlah baris dan kolom pada tabel credit_card_balance adalah : (3840312, 23)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [28]:
credit_card_balance.duplicated().sum()

0

In [29]:
credit_card_balance.isna().sum()

SK_ID_PREV                         0
SK_ID_CURR                         0
MONTHS_BALANCE                     0
AMT_BALANCE                        0
AMT_CREDIT_LIMIT_ACTUAL            0
AMT_DRAWINGS_ATM_CURRENT      749816
AMT_DRAWINGS_CURRENT               0
AMT_DRAWINGS_OTHER_CURRENT    749816
AMT_DRAWINGS_POS_CURRENT      749816
AMT_INST_MIN_REGULARITY       305236
AMT_PAYMENT_CURRENT           767988
AMT_PAYMENT_TOTAL_CURRENT          0
AMT_RECEIVABLE_PRINCIPAL           0
AMT_RECIVABLE                      0
AMT_TOTAL_RECEIVABLE               0
CNT_DRAWINGS_ATM_CURRENT      749816
CNT_DRAWINGS_CURRENT               0
CNT_DRAWINGS_OTHER_CURRENT    749816
CNT_DRAWINGS_POS_CURRENT      749816
CNT_INSTALMENT_MATURE_CUM     305236
NAME_CONTRACT_STATUS               0
SK_DPD                             0
SK_DPD_DEF                         0
dtype: int64

In [30]:
credit_card_balance['NAME_CONTRACT_STATUS'].value_counts()

NAME_CONTRACT_STATUS
Active           3698436
Completed         128918
Signed             11058
Demand              1365
Sent proposal        513
Refused               17
Approved               5
Name: count, dtype: int64

Proses merging ke previous_application

Untuk pos_cash

In [18]:
def simplify_pos_status(x):
    if x in ['Active', 'Signed', 'Approved']:
        return 'Active'
    elif x in ['Completed', 'Amortized debt']:
        return 'Completed'
    elif x in ['Demand', 'Returned to the store', 'Canceled']:
        return 'Problematic'
    else:
        return 'Unknown'

pos_cash['STATUS_SIMPLIFIED'] = pos_cash['NAME_CONTRACT_STATUS'].apply(simplify_pos_status)

In [19]:
pos_agg = pos_cash.groupby('SK_ID_PREV').agg(
    POS_RATIO_ACTIVE=('STATUS_SIMPLIFIED', lambda x: (x == 'Active').mean()),
    POS_RATIO_COMPLETED=('STATUS_SIMPLIFIED', lambda x: (x == 'Completed').mean()),
    POS_RATIO_PROBLEMATIC=('STATUS_SIMPLIFIED', lambda x: (x == 'Problematic').mean()),
    POS_AVG_INSTALMENT_FUTURE=('CNT_INSTALMENT_FUTURE', 'mean'),
    POS_MONTHS_COUNT=('MONTHS_BALANCE', 'count')
).reset_index()

Alasan tindakan dilakukan:
1. Menyederhanakan kategori agar fokus pada pola perilaku utama (aktif, selesai, atau bermasalah).
2. Untuk POS_RATIO_ACTIVE, akan mengukur proporsi waktu di mana pinjaman masih berjalan dengan dugaan semakin tinggi → nasabah cenderung menumpuk pinjaman aktif.
3. Untuk POS_RATIO_COMPLETED, akan menunjukkan kemampuan menyelesaikan kredit dengan dugaan rasio tinggi → sinyal good repayment history.
4. Untuk POS_RATIO_PROBLEMATIC, akan menunjukkan indikasi proporsi bulan bermasalah (Demand, Returned)
5. Untuk POS_AVG_INSTALMENT_FUTURE, lebih menggambarkan sisa tanggungan pinjaman dengan dugaan bahwa rata-rata CNT_INSTALMENT_FUTURE yang tinggi → banyak kredit belum lunas.
6. Untuk POS_MONTHS_COUNT, menggambarkan lama riwayat kredit POS (Riwayat panjang = stabilitas keuangan; singkat = risiko belum teruji.)

In [33]:
pos_agg.head()

,SK_ID_PREV,POS_RATIO_ACTIVE,POS_RATIO_COMPLETED,POS_RATIO_PROBLEMATIC,POS_AVG_INSTALMENT_FUTURE,POS_MONTHS_COUNT
0,1000001,0.666667,0.333333,0.0,7.666667,3
1,1000002,0.800000,0.200000,0.0,2.000000,5
2,1000003,1.000000,0.000000,0.0,10.500000,4
3,1000004,0.875000,0.125000,0.0,6.125000,8
4,1000005,0.909091,0.090909,0.0,5.000000,11


Untuk credit_card_balance

In [20]:
def simplify_cc_status(x):
    if x in ['Active', 'Signed', 'Approved', 'Sent proposal']:
        return 'Active'
    elif x == 'Completed':
        return 'Completed'
    elif x in ['Demand', 'Refused']:
        return 'Problematic'
    else:
        return 'Unknown'

credit_card_balance['STATUS_SIMPLIFIED'] = credit_card_balance['NAME_CONTRACT_STATUS'].apply(simplify_cc_status)

In [21]:
# Selaras dengan BigQuery: AVG(IFNULL(ratio, 0))
credit_card_balance['UTILIZATION'] = (
    credit_card_balance['AMT_BALANCE'] / credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL'].replace(0, np.nan)
).fillna(0)

credit_card_balance['PAYMENT_RATIO'] = (
    credit_card_balance['AMT_PAYMENT_CURRENT'] / credit_card_balance['AMT_DRAWINGS_CURRENT'].replace(0, np.nan)
).fillna(0)

In [22]:
cc_agg = credit_card_balance.groupby('SK_ID_PREV').agg(
    CC_RATIO_ACTIVE=('STATUS_SIMPLIFIED', lambda x: (x == 'Active').mean()),
    CC_RATIO_COMPLETED=('STATUS_SIMPLIFIED', lambda x: (x == 'Completed').mean()),
    CC_RATIO_PROBLEMATIC=('STATUS_SIMPLIFIED', lambda x: (x == 'Problematic').mean()),
    CC_UTILIZATION_MEAN=('UTILIZATION', 'mean'),
    CC_PAYMENT_RATIO_MEAN=('PAYMENT_RATIO', 'mean'),
    CC_AVG_DPD_DEF=('SK_DPD_DEF', lambda x: (x > 0).mean()),
    CC_AVG_PAYMENTS=('AMT_PAYMENT_TOTAL_CURRENT', 'mean'),
    CC_AVG_DRAWINGS=('AMT_DRAWINGS_CURRENT', 'mean'),
    CC_MONTHS_COUNT=('MONTHS_BALANCE', 'count')
).reset_index()

Alasan tindakan dilakukan:
1. Menyederhanakan kategori agar fokus pada pola perilaku utama (aktif, selesai, atau bermasalah).
2. Untuk CC_RATIO_(ACTIVE, COMPLETED, & PROBLEMATIC), sama seperti sebelumnya (Membantu membedakan nasabah aktif vs sudah menyelesaikan utang.)
3. Untuk CC_UTILIZATION_MEAN, digunakan untuk mengukur rata-rata pemakaian limit kartu.
4. Untuk CC_PAYMENT_RATIO_MEAN, menggambarkan rasio antara jumlah dibayar dengan jumlah ditarik.
5. Untuk CC_AVG_DPD_DEF, menyatakan proporsi bulan dengan keterlambatan (Days Past Due biasanya menjadi indikator kredit macet)
6. Untuk CC_AVG_(PAYMENTS & DRAWINGS), digunakan untuk mengukur tingkat intensitas transaksi & perilaku penggunaan (indikator gaya hidup finansial)
7. Untuk CC_MONTHS_COUNT (), menghitung lama waktu kartu kredit (dalam bulan)

In [41]:
cc_agg.sample(10)

,SK_ID_PREV,CC_RATIO_ACTIVE,CC_RATIO_COMPLETED,CC_RATIO_PROBLEMATIC,CC_UTILIZATION_MEAN,CC_PAYMENT_RATIO_MEAN,CC_AVG_DPD_DEF,CC_AVG_PAYMENTS,CC_AVG_DRAWINGS,CC_MONTHS_COUNT
32858,1545241,1.0,0.0,0.0,0.000000,NaN,0.000000,0.000000,0.000000,8
89487,2574955,1.0,0.0,0.0,0.747898,6.346023,0.120482,10203.614458,8773.200542,83
86573,2524453,1.0,0.0,0.0,0.915136,0.398333,0.000000,26212.000000,32500.000000,18
4692,1078961,1.0,0.0,0.0,0.248062,2.419048,0.000000,1735.078125,356.250000,96
34745,1575908,1.0,0.0,0.0,0.303825,0.039163,0.000000,29455.976250,37383.750000,12
84139,2478158,1.0,0.0,0.0,0.000000,NaN,0.000000,0.000000,0.000000,15
31946,1532183,1.0,0.0,0.0,0.000000,NaN,0.000000,0.000000,0.000000,6
53290,1920616,1.0,0.0,0.0,0.132028,0.746363,0.010526,5660.743263,3833.857895,95
29295,1485077,1.0,0.0,0.0,0.751176,0.855185,0.229167,4543.939219,2690.625000,96
34037,1566385,1.0,0.0,0.0,0.519993,0.357143,0.000000,4601.476250,5435.000000,36


Untuk installments_payments

In [23]:
installments_payments['PAYMENT_RATIO'] = installments_payments['AMT_PAYMENT'] / installments_payments['AMT_INSTALMENT'].replace(0, np.nan)
installments_payments['IS_LATE'] = (installments_payments['DAYS_ENTRY_PAYMENT'] - installments_payments['DAYS_INSTALMENT']) > 0
installments_payments['LATE_DAYS'] = (installments_payments['DAYS_ENTRY_PAYMENT'] - installments_payments['DAYS_INSTALMENT']).clip(lower=0)

In [24]:
inst_agg = installments_payments.groupby('SK_ID_PREV').agg(
    INST_PAYMENT_RATIO_MEAN=('PAYMENT_RATIO', 'mean'),
    INST_LATE_PAYMENT_RATIO=('IS_LATE', 'mean'),
    INST_AVG_DELAY_DAYS=('LATE_DAYS', 'mean'),
    INST_MISSED_PAYMENT_RATE=('PAYMENT_RATIO', lambda x: (x < 1).mean()),
    INST_AVG_PAYMENT_AMT=('AMT_PAYMENT', 'mean')
).reset_index()

Alasan tindakan dilakukan:
1. Untuk 'INST_PAYMENT_RATIO_MEAN', rata-rata tingkat pembayaran terhadap kewajiban
2. Untuk 'INST_LATE_PAYMENT_RATIO', proporsi pembayaran cicilan yang terlambat
3. Untuk 'AVG_DELAY_DAYS', rata-rata hari keterlambatan membayar aktual
4. Untuk 'INST_MISSED_PAYMENT_RATE', proporsi pembayaran di bawah kewajiban
5. Untuk 'INST_AVG_PAYMENT_AMT', rata-rata nominal pembayaran

In [42]:
inst_agg.sample(10)

,SK_ID_PREV,INST_PAYMENT_RATIO_MEAN,INST_LATE_PAYMENT_RATIO,INST_AVG_DELAY_DAYS,INST_MISSED_PAYMENT_RATE,INST_AVG_PAYMENT_AMT
919365,2697974,1.000000,0.000000,0.000000,0.000000,10939.410000
723777,2318868,1.000000,0.000000,0.000000,0.000000,12778.120000
645719,2174407,1.000000,0.000000,0.000000,0.000000,21698.430000
457688,1818186,0.857143,0.142857,0.285714,0.285714,2456.800714
163599,1285450,0.545455,0.545455,11.909091,0.909091,2061.045000
599786,2085434,1.000000,0.000000,0.000000,0.000000,32961.564000
263070,1455873,0.800000,0.200000,1.600000,0.400000,26624.988000
215494,1374636,1.584060,0.038462,0.153846,0.076923,89329.840962
595956,2078048,1.000000,0.000000,0.000000,0.000000,9668.010000
565089,2017375,1.000000,0.000000,0.000000,0.000000,6540.660000


Menggabungkan semuanya ke previous_application

In [25]:
prev_application = pd.read_csv('previous_application.csv')
print(f'Jumlah baris dan kolom pada tabel previous_application adalah : {prev_application.shape}')
prev_application.head()

Jumlah baris dan kolom pada tabel previous_application adalah : (1670214, 37)


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
prev_application_merged = (
    prev_application
    .merge(pos_agg, on='SK_ID_PREV', how='left')
    .merge(cc_agg, on='SK_ID_PREV', how='left')
    .merge(inst_agg, on='SK_ID_PREV', how='left')
)

In [52]:
prev_application_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 56 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   SK_ID_PREV                   1670214 non-null  int64  
 1   SK_ID_CURR                   1670214 non-null  int64  
 2   NAME_CONTRACT_TYPE           1670214 non-null  object 
 3   AMT_ANNUITY                  1297979 non-null  float64
 4   AMT_APPLICATION              1670214 non-null  float64
 5   AMT_CREDIT                   1670213 non-null  float64
 6   AMT_DOWN_PAYMENT             774370 non-null   float64
 7   AMT_GOODS_PRICE              1284699 non-null  float64
 8   WEEKDAY_APPR_PROCESS_START   1670214 non-null  object 
 9   HOUR_APPR_PROCESS_START      1670214 non-null  int64  
 10  FLAG_LAST_APPL_PER_CONTRACT  1670214 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY       1670214 non-null  int64  
 12  RATE_DOWN_PAYMENT            774370 non-nu

In [51]:
missing_threshold = 0.4
missing_data_prev = prev_application_merged.isnull().sum()
missing_data_prev = missing_data_prev[missing_data_prev > missing_threshold * len(prev_application_merged)]
print(f"Jumlah kolom dengan missing value lebih dari 40% pada previous_application_merged: {len(missing_data_prev)}")
missing_data_prev


Jumlah kolom dengan missing value lebih dari 40% pada previous_application_merged: 30


AMT_DOWN_PAYMENT              895844
RATE_DOWN_PAYMENT             895844
RATE_INTEREST_PRIMARY        1664263
RATE_INTEREST_PRIVILEGED     1664263
NAME_TYPE_SUITE               820405
DAYS_FIRST_DRAWING            673065
DAYS_FIRST_DUE                673065
DAYS_LAST_DUE_1ST_VERSION     673065
DAYS_LAST_DUE                 673065
DAYS_TERMINATION              673065
NFLAG_INSURED_ON_APPROVAL     673065
POS_RATIO_ACTIVE              771311
POS_RATIO_COMPLETED           771311
POS_RATIO_PROBLEMATIC         771311
POS_AVG_INSTALMENT_FUTURE     772198
POS_MONTHS_COUNT              771311
CC_RATIO_ACTIVE              1577279
CC_RATIO_COMPLETED           1577279
CC_RATIO_PROBLEMATIC         1577279
CC_UTILIZATION_MEAN          1578452
CC_PAYMENT_RATIO_MEAN        1610349
CC_AVG_DPD_DEF               1577279
CC_AVG_PAYMENTS              1577279
CC_AVG_DRAWINGS              1577279
CC_MONTHS_COUNT              1577279
INST_PAYMENT_RATIO_MEAN       711389
INST_LATE_PAYMENT_RATIO       711309
I

##### Fitur previous_application yang akan dimerge ke Tabel Utama

In [27]:
selected_features = [
    'SK_ID_CURR', 'SK_ID_PREV',
    'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
    'DAYS_DECISION', 'CNT_PAYMENT', 'NAME_CONTRACT_STATUS', 'NAME_CONTRACT_TYPE',
    'POS_RATIO_ACTIVE', 'POS_RATIO_COMPLETED', 'POS_RATIO_PROBLEMATIC',
    'CC_UTILIZATION_MEAN', 'CC_AVG_DPD_DEF',
    'INST_PAYMENT_RATIO_MEAN', 'INST_LATE_PAYMENT_RATIO', 'INST_MISSED_PAYMENT_RATE',
    'NAME_PORTFOLIO', 'NAME_CLIENT_TYPE', 'NAME_PRODUCT_TYPE', 'CODE_REJECT_REASON'
]

prev_summary = prev_application_merged[selected_features].copy()

In [34]:
# --- Imputasi & fitur fixed (selaras dengan ETL/BigQuery)
prev_summary = prev_summary.copy()

# numerik utama
prev_summary["AMT_ANNUITY_IMP"] = prev_summary["AMT_ANNUITY"].fillna(prev_summary["AMT_ANNUITY"].median())
prev_summary["AMT_GOODS_PRICE_IMP"] = prev_summary["AMT_GOODS_PRICE"].fillna(prev_summary["AMT_GOODS_PRICE"].median())
prev_summary["CNT_PAYMENT_IMP"] = prev_summary["CNT_PAYMENT"].fillna(prev_summary["CNT_PAYMENT"].median())

# POS (NaN => 0)
prev_summary["POS_RATIO_ACTIVE_IMP"] = prev_summary["POS_RATIO_ACTIVE"].fillna(0)
prev_summary["POS_RATIO_COMPLETED_IMP"] = prev_summary["POS_RATIO_COMPLETED"].fillna(0)
prev_summary["POS_RATIO_PROBLEMATIC_IMP"] = prev_summary["POS_RATIO_PROBLEMATIC"].fillna(0)

# CC (NaN => 0)
prev_summary["CC_UTILIZATION_MEAN_IMP"] = prev_summary["CC_UTILIZATION_MEAN"].fillna(0)
prev_summary["CC_AVG_DPD_DEF_IMP"] = prev_summary["CC_AVG_DPD_DEF"].fillna(0)

# INST (NaN => stabil)
prev_summary["INST_PAYMENT_RATIO_MEAN_IMP"] = prev_summary["INST_PAYMENT_RATIO_MEAN"].fillna(1)
prev_summary["INST_LATE_PAYMENT_RATIO_IMP"] = prev_summary["INST_LATE_PAYMENT_RATIO"].fillna(0)
prev_summary["INST_MISSED_PAYMENT_RATE_IMP"] = prev_summary["INST_MISSED_PAYMENT_RATE"].fillna(0)

# kategorikal (NaN => 'Unknown')
prev_summary["NAME_CONTRACT_STATUS_IMP"] = prev_summary["NAME_CONTRACT_STATUS"].fillna("Unknown")
prev_summary["NAME_CONTRACT_TYPE_IMP"] = prev_summary["NAME_CONTRACT_TYPE"].fillna("Unknown")
prev_summary["NAME_PORTFOLIO_IMP"] = prev_summary["NAME_PORTFOLIO"].fillna("Unknown")
prev_summary["NAME_CLIENT_TYPE_IMP"] = prev_summary["NAME_CLIENT_TYPE"].fillna("Unknown")
prev_summary["NAME_PRODUCT_TYPE_IMP"] = prev_summary["NAME_PRODUCT_TYPE"].fillna("Unknown")
prev_summary["CODE_REJECT_REASON_IMP"] = prev_summary["CODE_REJECT_REASON"].fillna("Unknown")

# fixed dummy-as-proportion columns (nilai per-row, nanti di-mean saat groupby)
prev_summary["PREV_CONTRACT_STATUS_APPROVED_RATIO"] = (prev_summary["NAME_CONTRACT_STATUS_IMP"] == "Approved").astype(int)
prev_summary["PREV_CONTRACT_STATUS_REFUSED_RATIO"] = (prev_summary["NAME_CONTRACT_STATUS_IMP"] == "Refused").astype(int)
prev_summary["PREV_CONTRACT_STATUS_CANCELED_RATIO"] = (prev_summary["NAME_CONTRACT_STATUS_IMP"] == "Canceled").astype(int)
prev_summary["PREV_CONTRACT_STATUS_UNUSED_OFFER_RATIO"] = (prev_summary["NAME_CONTRACT_STATUS_IMP"] == "Unused offer").astype(int)
prev_summary["PREV_CONTRACT_STATUS_UNKNOWN_RATIO"] = (prev_summary["NAME_CONTRACT_STATUS_IMP"] == "Unknown").astype(int)

prev_summary["PREV_CONTRACT_TYPE_CASH_LOANS_RATIO"] = (prev_summary["NAME_CONTRACT_TYPE_IMP"] == "Cash loans").astype(int)
prev_summary["PREV_CONTRACT_TYPE_CONSUMER_LOANS_RATIO"] = (prev_summary["NAME_CONTRACT_TYPE_IMP"] == "Consumer loans").astype(int)
prev_summary["PREV_CONTRACT_TYPE_REVOLVING_LOANS_RATIO"] = (prev_summary["NAME_CONTRACT_TYPE_IMP"] == "Revolving loans").astype(int)
prev_summary["PREV_CONTRACT_TYPE_UNKNOWN_RATIO"] = (prev_summary["NAME_CONTRACT_TYPE_IMP"] == "Unknown").astype(int)

prev_summary["PREV_PORTFOLIO_CASH_RATIO"] = (prev_summary["NAME_PORTFOLIO_IMP"] == "Cash").astype(int)
prev_summary["PREV_PORTFOLIO_CARDS_RATIO"] = (prev_summary["NAME_PORTFOLIO_IMP"] == "Cards").astype(int)
prev_summary["PREV_PORTFOLIO_POS_RATIO"] = (prev_summary["NAME_PORTFOLIO_IMP"] == "POS").astype(int)
prev_summary["PREV_PORTFOLIO_CARS_RATIO"] = (prev_summary["NAME_PORTFOLIO_IMP"] == "Cars").astype(int)
prev_summary["PREV_PORTFOLIO_UNKNOWN_RATIO"] = (prev_summary["NAME_PORTFOLIO_IMP"] == "Unknown").astype(int)

prev_summary["PREV_CLIENT_TYPE_NEW_RATIO"] = (prev_summary["NAME_CLIENT_TYPE_IMP"] == "New").astype(int)
prev_summary["PREV_CLIENT_TYPE_REPEATER_RATIO"] = (prev_summary["NAME_CLIENT_TYPE_IMP"] == "Repeater").astype(int)
prev_summary["PREV_CLIENT_TYPE_REFRESHED_RATIO"] = (prev_summary["NAME_CLIENT_TYPE_IMP"] == "Refreshed").astype(int)
prev_summary["PREV_CLIENT_TYPE_UNKNOWN_RATIO"] = (prev_summary["NAME_CLIENT_TYPE_IMP"] == "Unknown").astype(int)

prev_summary["PREV_PRODUCT_TYPE_WALKIN_RATIO"] = (prev_summary["NAME_PRODUCT_TYPE_IMP"] == "walk-in").astype(int)
prev_summary["PREV_PRODUCT_TYPE_XSELL_RATIO"] = (prev_summary["NAME_PRODUCT_TYPE_IMP"] == "x-sell").astype(int)
prev_summary["PREV_PRODUCT_TYPE_UNKNOWN_RATIO"] = (prev_summary["NAME_PRODUCT_TYPE_IMP"] == "Unknown").astype(int)

# Fixed reject-reason ratios (note: last one must be PREV_REJECT_UNKNOWN_RATIO)
for code in ["XAP", "XNA", "HC", "LIMIT", "SCO", "SCOFR", "SYSTEM", "VERIF"]:
    prev_summary[f"PREV_REJECT_{code}_RATIO"] = (prev_summary["CODE_REJECT_REASON_IMP"] == code).astype(int)
prev_summary["PREV_REJECT_UNKNOWN_RATIO"] = (prev_summary["CODE_REJECT_REASON_IMP"] == "Unknown").astype(int)

In [ ]:
# NOTE: untuk menyamakan jumlah kolom dengan ETL, kita tidak memakai get_dummies dinamis.
# Kolom rasio fixed (PREV_..._RATIO) sudah dibuat pada sel sebelumnya.

C:\Users\hardi\AppData\Local\Temp\ipykernel_11472\3186360066.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  prev_summary['CNT_PAYMENT'].fillna(prev_summary['CNT_PAYMENT'].median(), inplace=True)
C:\Users\hardi\AppData\Local\Temp\ipykernel_11472\3186360066.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

In [64]:
prev_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 39 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   SK_ID_CURR                          1670214 non-null  int64  
 1   SK_ID_PREV                          1670214 non-null  int64  
 2   AMT_APPLICATION                     1670214 non-null  float64
 3   AMT_CREDIT                          1670213 non-null  float64
 4   AMT_ANNUITY                         1670214 non-null  float64
 5   AMT_GOODS_PRICE                     1670214 non-null  float64
 6   DAYS_DECISION                       1670214 non-null  int64  
 7   CNT_PAYMENT                         1670214 non-null  float64
 8   POS_RATIO_ACTIVE                    1670214 non-null  float64
 9   POS_RATIO_COMPLETED                 1670214 non-null  float64
 10  POS_RATIO_PROBLEMATIC               1670214 non-null  float64
 11  CC_UTILIZAT

In [65]:
prev_summary.sample(10)

,SK_ID_CURR,SK_ID_PREV,AMT_APPLICATION,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_DECISION,CNT_PAYMENT,POS_RATIO_ACTIVE,POS_RATIO_COMPLETED,POS_RATIO_PROBLEMATIC,CC_UTILIZATION_MEAN,CC_AVG_DPD_DEF,INST_PAYMENT_RATIO_MEAN,INST_LATE_PAYMENT_RATIO,INST_MISSED_PAYMENT_RATE,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Unused offer,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,NAME_PORTFOLIO_Cars,NAME_PORTFOLIO_Cash,NAME_PORTFOLIO_POS,NAME_PORTFOLIO_XNA,NAME_CLIENT_TYPE_Refreshed,NAME_CLIENT_TYPE_Repeater,NAME_CLIENT_TYPE_XNA,NAME_PRODUCT_TYPE_walk-in,NAME_PRODUCT_TYPE_x-sell,CODE_REJECT_REASON_HC,CODE_REJECT_REASON_LIMIT,CODE_REJECT_REASON_SCO,CODE_REJECT_REASON_SCOFR,CODE_REJECT_REASON_SYSTEM,CODE_REJECT_REASON_VERIF,CODE_REJECT_REASON_XAP,CODE_REJECT_REASON_XNA
1024754,190949,1306031,72632.25,58104.0,10321.020,72632.25,-848,6.0,0.750000,0.250000,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0
1596835,358638,1576884,1125000.00,1223010.0,58839.030,1125000.00,-841,36.0,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0
1561106,384581,1853532,43155.00,38839.5,4532.580,43155.00,-1505,10.0,0.909091,0.090909,0.0,0.0,0.0,1.0,0.1,0.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
722526,373298,2279869,225000.00,225000.0,22482.000,225000.00,-1257,12.0,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0
1591470,122874,2056558,29817.00,25317.0,5164.650,29817.00,-2082,6.0,0.857143,0.142857,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
991403,231259,1242870,225000.00,348700.5,15792.660,225000.00,-818,48.0,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0
1106974,377484,1035262,45000.00,45000.0,2250.000,45000.00,-270,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
97593,250675,1446578,225000.00,247275.0,19151.460,225000.00,-1047,18.0,0.857143,0.142857,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
1150634,231885,2276038,152122.50,152122.5,7860.150,152122.50,-835,24.0,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0
1628929,375476,2326718,450000.00,521280.0,19986.075,450000.00,-330,36.0,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0


In [38]:
agg_map = {
    "SK_ID_PREV": "count",
    "AMT_APPLICATION": "mean",
    "AMT_CREDIT": "mean",
    "AMT_ANNUITY_IMP": "mean",
    "AMT_GOODS_PRICE_IMP": "mean",
    "DAYS_DECISION": "mean",
    "CNT_PAYMENT_IMP": "mean",
    "POS_RATIO_ACTIVE_IMP": "mean",
    "POS_RATIO_COMPLETED_IMP": "mean",
    "POS_RATIO_PROBLEMATIC_IMP": "mean",
    "CC_UTILIZATION_MEAN_IMP": "mean",
    "CC_AVG_DPD_DEF_IMP": "mean",
    "INST_PAYMENT_RATIO_MEAN_IMP": "mean",
    "INST_LATE_PAYMENT_RATIO_IMP": "mean",
    "INST_MISSED_PAYMENT_RATE_IMP": "mean",
}

ratio_cols = [c for c in prev_summary.columns if c.startswith("PREV_") and c.endswith("_RATIO")]
for c in ratio_cols:
    agg_map[c] = "mean"

prev_summary_agg = prev_summary.groupby("SK_ID_CURR").agg(agg_map).reset_index()

prev_summary_agg = prev_summary_agg.rename(columns={
    "SK_ID_PREV": "PREV_APP_COUNT",
    "AMT_APPLICATION": "PREV_AMT_APPLICATION_MEAN",
    "AMT_CREDIT": "PREV_AMT_CREDIT_MEAN",
    "AMT_ANNUITY_IMP": "PREV_AMT_ANNUITY_MEAN",
    "AMT_GOODS_PRICE_IMP": "PREV_AMT_GOODS_PRICE_MEAN",
    "DAYS_DECISION": "PREV_DAYS_DECISION_MEAN",
    "CNT_PAYMENT_IMP": "PREV_CNT_PAYMENT_MEAN",
    "POS_RATIO_ACTIVE_IMP": "PREV_POS_RATIO_ACTIVE_MEAN",
    "POS_RATIO_COMPLETED_IMP": "PREV_POS_RATIO_COMPLETED_MEAN",
    "POS_RATIO_PROBLEMATIC_IMP": "PREV_POS_RATIO_PROBLEMATIC_MEAN",
    "CC_UTILIZATION_MEAN_IMP": "PREV_CC_UTILIZATION_MEAN",
    "CC_AVG_DPD_DEF_IMP": "PREV_CC_AVG_DPD_DEF_MEAN",
    "INST_PAYMENT_RATIO_MEAN_IMP": "PREV_INST_PAYMENT_RATIO_MEAN",
    "INST_LATE_PAYMENT_RATIO_IMP": "PREV_INST_LATE_PAYMENT_RATIO_MEAN",
    "INST_MISSED_PAYMENT_RATE_IMP": "PREV_INST_MISSED_PAYMENT_RATE_MEAN",
})

In [67]:
prev_summary_agg.sample(10)

,SK_ID_CURR,AMT_APPLICATION,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_DECISION,CNT_PAYMENT,POS_RATIO_ACTIVE,POS_RATIO_COMPLETED,POS_RATIO_PROBLEMATIC,CC_UTILIZATION_MEAN,CC_AVG_DPD_DEF,INST_PAYMENT_RATIO_MEAN,INST_LATE_PAYMENT_RATIO,INST_MISSED_PAYMENT_RATE,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Unused offer,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,NAME_PORTFOLIO_Cars,NAME_PORTFOLIO_Cash,NAME_PORTFOLIO_POS,NAME_PORTFOLIO_XNA,NAME_CLIENT_TYPE_Refreshed,NAME_CLIENT_TYPE_Repeater,NAME_CLIENT_TYPE_XNA,NAME_PRODUCT_TYPE_walk-in,NAME_PRODUCT_TYPE_x-sell,CODE_REJECT_REASON_HC,CODE_REJECT_REASON_LIMIT,CODE_REJECT_REASON_SCO,CODE_REJECT_REASON_SCOFR,CODE_REJECT_REASON_SYSTEM,CODE_REJECT_REASON_VERIF,CODE_REJECT_REASON_XAP,CODE_REJECT_REASON_XNA
218647,329997,48759.750,46244.25,5024.2725,48759.750,-1459.000000,12.000000,0.961538,0.038462,0.0,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.000000,1.000000,0.000000,0.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.0
179591,288876,175070.025,185048.25,23469.8250,175070.025,-1076.000000,6.000000,0.483135,0.183532,0.0,0.021499,0.0,1.000000,0.166667,0.000000,0.000000,0.166667,0.000000,0.666667,0.166667,0.0,0.0,0.166667,0.666667,0.000000,0.166667,0.666667,0.0,0.000000,0.333333,0.166667,0.000000,0.0,0.000000,0.0,0.0,0.833333,0.0
253887,367032,45378.900,38070.90,9437.4000,59418.900,-1168.000000,10.000000,0.216518,0.033482,0.0,0.000000,0.0,1.000000,0.750000,0.000000,0.125000,0.625000,0.000000,0.875000,0.000000,0.0,0.0,0.000000,0.500000,0.500000,0.000000,0.875000,0.0,0.000000,0.000000,0.000000,0.625000,0.0,0.000000,0.0,0.0,0.375000,0.0
194348,304428,378255.000,413989.50,21066.1200,415695.000,-633.000000,27.333333,0.305556,0.027778,0.0,0.000000,0.0,0.916667,0.791667,0.166667,0.333333,0.333333,0.000000,0.333333,0.000000,0.0,0.0,0.333333,0.333333,0.333333,0.000000,0.666667,0.0,0.333333,0.000000,0.333333,0.000000,0.0,0.000000,0.0,0.0,0.666667,0.0
198371,308678,69663.750,63883.50,8258.0700,88383.750,-1567.333333,12.333333,0.481481,0.018519,0.0,0.000000,0.0,0.980000,0.526667,0.040000,0.166667,0.000000,0.166667,0.833333,0.000000,0.0,0.0,0.000000,0.666667,0.333333,0.500000,0.166667,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.833333,0.0
331232,448268,95400.000,95400.00,6792.4800,95400.000,-2324.000000,18.000000,1.000000,0.000000,0.0,0.000000,0.0,1.000000,0.055556,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.0
146140,253698,74133.000,83740.50,6084.7650,74133.000,-1050.000000,18.000000,0.937322,0.062678,0.0,0.000000,0.0,0.979167,0.069444,0.041667,0.000000,0.000000,0.000000,0.666667,0.000000,0.0,0.0,0.333333,0.666667,0.000000,0.666667,0.333333,0.0,0.000000,0.333333,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.0
264840,378437,111160.575,111600.00,10143.4050,148600.575,-775.000000,18.000000,0.454598,0.045402,0.0,0.000000,0.0,0.987179,0.551282,0.025641,0.166667,0.333333,0.000000,0.666667,0.000000,0.0,0.0,0.000000,0.666667,0.333333,0.000000,0.833333,0.0,0.000000,0.000000,0.000000,0.166667,0.0,0.166667,0.0,0.0,0.666667,0.0
157559,265719,348750.000,384903.75,37584.9450,367470.000,-850.000000,12.000000,0.467692,0.032308,0.0,0.157863,0.0,0.938421,0.556343,0.123157,0.166667,0.166667,0.000000,0.000000,0.166667,0.0,0.0,0.666667,0.000000,0.166667,0.166667,0.833333,0.0,0.333333,0.500000,0.166667,0.000000,0.0,0.000000,0.0,0.0,0.833333,0.0
135,100145,232737.900,263916.00,14045.7150,232737.900,-1704.833333,22.000000,0.776515,0.056818,0.0,0.000000,0.0,1.000000,0.166667,0.000000,0.000000,0.166667,0.000000,0.666667,0.000000,0.0,0.0,0.333333,0.666667,0.000000,0.000000,1.000000,0.0,0.000000,0.333333,0.000000,0.166667,0.0,0.000000,0.0,0.0,0.833333,0.0


##### Merging semuanya ke tabel utama

In [36]:
new_application_data_insight = (
    main_table
    .merge(bureau_summary, on='SK_ID_CURR', how='left')
    .merge(prev_summary_agg, on='SK_ID_CURR', how='left')
)

In [31]:
print(f'new_application_data_insight shape: {new_application_data_insight.shape}')
new_application_data_insight.sample(30)

new_application_data_insight shape: (307511, 181)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,STATUS_ACTIVE,STATUS_CLOSED,STATUS_BAD_DEBT,TOTAL_CREDITS,RATIO_BAD_TO_TOTAL,CREDIT_GROUP_RETAIL,CREDIT_GROUP_ASSET,CREDIT_GROUP_BUSINESS,RATIO_ASSET_TO_TOTAL,AMT_CREDIT_SUM_MEAN,AMT_CREDIT_SUM_DEBT_MEAN,RATIO_DEBT_TO_CREDIT_MEAN,DAYS_CREDIT_MEAN,DAYS_CREDIT_ENDDATE_MEAN,PREV_APP_COUNT,PREV_AMT_APPLICATION_MEAN,PREV_AMT_CREDIT_MEAN,PREV_AMT_ANNUITY_MEAN,PREV_AMT_GOODS_PRICE_MEAN,PREV_DAYS_DECISION_MEAN,PREV_CNT_PAYMENT_MEAN,PREV_POS_RATIO_ACTIVE_MEAN,PREV_POS_RATIO_COMPLETED_MEAN,PREV_POS_RATIO_PROBLEMATIC_MEAN,PREV_CC_UTILIZATION_MEAN,PREV_CC_AVG_DPD_DEF_MEAN,PREV_INST_PAYMENT_RATIO_MEAN,PREV_INST_LATE_PAYMENT_RATIO_MEAN,PREV_INST_MISSED_PAYMENT_RATE_MEAN,PREV_CONTRACT_STATUS_APPROVED_RATIO,PREV_CONTRACT_STATUS_REFUSED_RATIO,PREV_CONTRACT_STATUS_CANCELED_RATIO,PREV_CONTRACT_STATUS_UNUSED_OFFER_RATIO,PREV_CONTRACT_STATUS_UNKNOWN_RATIO,PREV_CONTRACT_TYPE_CASH_LOANS_RATIO,PREV_CONTRACT_TYPE_CONSUMER_LOANS_RATIO,PREV_CONTRACT_TYPE_REVOLVING_LOANS_RATIO,PREV_CONTRACT_TYPE_UNKNOWN_RATIO,PREV_PORTFOLIO_CASH_RATIO,PREV_PORTFOLIO_CARDS_RATIO,PREV_PORTFOLIO_POS_RATIO,PREV_PORTFOLIO_CARS_RATIO,PREV_PORTFOLIO_UNKNOWN_RATIO,PREV_CLIENT_TYPE_NEW_RATIO,PREV_CLIENT_TYPE_REPEATER_RATIO,PREV_CLIENT_TYPE_REFRESHED_RATIO,PREV_CLIENT_TYPE_UNKNOWN_RATIO,PREV_PRODUCT_TYPE_WALKIN_RATIO,PREV_PRODUCT_TYPE_XSELL_RATIO,PREV_PRODUCT_TYPE_UNKNOWN_RATIO,PREV_REJECT_XAP_RATIO,PREV_REJECT_XNA_RATIO,PREV_REJECT_HC_RATIO,PREV_REJECT_LIMIT_RATIO,PREV_REJECT_SCO_RATIO,PREV_REJECT_SCOFR_RATIO,PREV_REJECT_SYSTEM_RATIO,PREV_REJECT_VERIF_RATIO,PREV_REJECT_Unknown_RATIO
226615,362495,0,Cash loans,M,Y,N,1,315000.0,1512000.0,41580.0,1512000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.030755,-11891,-621,-1206.0,-4456,1.0,1,1,1,1,0,0,Sales staff,3.0,2,2,TUESDAY,14,0,0,0,0,0,

In [40]:
new_application_data_insight.to_csv('merging_result_for_analysis.csv', index=False)